## Import

In [2]:
from tqdm import tqdm

import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import dart_fss
import time
import re, os

warnings.filterwarnings('ignore')

* functions

In [3]:
def get_data(dart, code, year = 2022, quarter = "사업보고서"):
    select_cols = ['corp_cls', 'corp_code', 'corp_name', 'inv_prm', 'frst_acqs_de', 'invstmnt_purps', 'frst_acqs_amount', 'trmend_blce_qy', 'trmend_blce_qota_rt', 'trmend_blce_acntbk_amount']
    change_cols = ['법인구분', '고유번호', '회사명', '법인명', '최초취득일자', '출자목적', '최초취득금액', '기말잔액수량', '기말잔액지분율', '기말잔액장부가액']
    change_cls = {"Y":"유가", "K":"코스닥", "N":"코넥스", "E":"기타"}
    
    change_dict = {"1분기보고서": 11013, "반기보고서": 11012, "3분기보고서": 11014, "사업보고서":11011}
    r_code = change_dict[quarter]
    
    invst_df = dart.report(code, '타법인출자', year, r_code)
    
    if invst_df.shape[0] == 0:
        return invst_df
    else:
        invst_df = invst_df.loc[:, select_cols]
        invst_df.corp_cls = invst_df.corp_cls.map(change_cls)
        invst_df.columns = change_cols
    
        return invst_df

* API Setting

In [4]:
api_key = '1b39652cef07f626c9d37375edf582ee51b1407f'
dart = OpenDartReader(api_key)
dart_fss.set_api_key(api_key=api_key)

'1b39652cef07f626c9d37375edf582ee51b1407f'

* get total corp list

In [5]:
corp_dict = dart_fss.api.filings.get_corp_code()
corp_df = pd.DataFrame(corp_dict)
corp_df = corp_df.loc[corp_df.stock_code.notnull()]
corp_df.index = [x for x in range(corp_df.shape[0])]

Output()

In [6]:
# C=발행공시, D=지분공시
market_dict = {"Y": "코스피","K": "코스닥", "N": "코넥스", "E": "기타"}
q_dict = {"1분기": ['{}-01-01', '{}-03-31'],
          "2분기": ['{}-04-01', '{}-06-30'],
          "3분기": ['{}-07-01', '{}-09-30'],
          "4분기": ['{}-10-01', '{}-12-31']}
change_dict = {"1분기보고서": 11013, "반기보고서": 11012, "3분기보고서": 11014, "사업보고서":11011}
year = 2022
r_code = '사업보고서'

In [7]:
loop_list = ['{}분기'.format(x) for x in range(1,5)]

In [8]:
for idx, q in enumerate(loop_list):
    start_dt, end_dt = [x.format(year) for x in q_dict[q]]
    
    if r_code == "사업보고서":
        temp_df = dart.list(start=start_dt, end=end_dt, kind_detail = 'A001') # 사업
    elif r_code == "반기보고서":
        temp_df = dart.list(start=start_dt, end=end_dt, kind_detail = 'A002') # 반기
    else:
        temp_df = dart.list(start=start_dt, end=end_dt, kind_detail = 'A003') # 분기
    
    if idx == 0:
        info_df = temp_df
    else:
        info_df = pd.concat([info_df, temp_df])

In [9]:
corp_list = info_df.loc[info_df.corp_cls.isin(['Y', 'K', 'N'])].corp_name.unique()

In [10]:
cnt = 0
p_cnt = 0

for corp in tqdm(corp_list):
    temp_df = dart.report(corp, '타법인출자', year, change_dict[r_code])
    p_cnt += 1
    
    if temp_df.shape[0] == 0:
        continue
    elif cnt == 0 and temp_df.shape[0] != 1:
        output = temp_df
        cnt += 1
    else:
        output = pd.concat([output, temp_df])

  0%|                                                                                 | 3/2428 [00:00<04:37,  8.75it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


  0%|▏                                                                                | 6/2428 [00:00<04:25,  9.13it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


  0%|▏                                                                                | 7/2428 [00:00<04:30,  8.97it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████▋| 2419/2428 [05:01<00:01,  8.36it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


100%|█████████████████████████████████████████████████████████████████████████████▋| 2420/2428 [05:01<00:00,  8.30it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


100%|██████████████████████████████████████████████████████████████████████████████| 2428/2428 [05:03<00:00,  8.01it/s]


In [11]:
output.invstmnt_purps = [x.replace(" ", "").replace("\n", "") for x in output.invstmnt_purps]
final_df = output.loc[[True if "단순투자" in x else False for x in output.invstmnt_purps], :]

In [13]:
select_cols = ['corp_cls', 'corp_name', 'inv_prm', 'frst_acqs_de', 'invstmnt_purps', 'frst_acqs_amount', 'trmend_blce_qy', 'trmend_blce_qota_rt', 'trmend_blce_acntbk_amount']
change_cols = ['법인구분', '회사명', '법인명', '최초취득일자', '출자목적', '최초취득금액', '기말잔액수량', '기말잔액지분율', '기말잔액장부가액']

In [ ]:
save_df = final_df.loc[:, select_cols]
save_df.columns = change_cols

save_df['법인구분'] = save_df['법인구분'].map(market_dict)

clean_txt = [re.sub(r'\([^)]*\)', '', x) for x in save_df['법인명']]
clean_txt = [re.sub("㈜", "", x) for x in clean_txt]
clean_txt = [re.sub("\s", "", x) for x in clean_txt]
save_df['법인명'] = clean_txt

check = save_df.loc[save_df['법인명'].isin(list(corp_df.corp_name.unique()))]
check.loc[check['회사명'] == '테라젠이텍스']

In [21]:
check.to_excel("ECM2부-타법인출자현황-단순투자-2022-사업보고서.xlsx", index = False, encoding = "CP949")